In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample



In [6]:
df=pd.read_excel(r'C:\Users\hp\Desktop\M2\PFE\Code\code pfe\dataset\6.xlsx')
df.shape

(17280, 42)

In [7]:
df['index']=pd.to_datetime(df['index'])

In [8]:
import pandas as pd
from sklearn.utils import resample

# Create a DataFrame with features X and labels y
X = df[['index', 'TVA (m3)', 'SPPA (kPa)', 'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)']]
y = df['STATUS']




In [11]:
# Concatenate X and y for easier resampling
df_concat = pd.concat([X, y], axis=1)

# Separate majority and minority class samples
majority_class = df_concat[df_concat['STATUS'] == 0]
minority_class = df_concat[(df_concat['STATUS'] == 1) & (df_concat.index >= 14571) & (df_concat.index <= 14679)]

# Resample the minority class sequences to match the length of the majority class sequences
resampled_minority_class = resample(minority_class,
                                    replace=True,  # Allow replacement for oversampling
                                    n_samples=len(majority_class),  # Match majority class samples
                                    random_state=42)

# Combine resampled minority class with majority class
balanced_df = pd.concat([majority_class, resampled_minority_class])

# Sort the DataFrame by the time column to maintain the time-series order
balanced_df.sort_values('index', inplace=True)

# Separate features (X) and labels (y) in the balanced dataset
balanced_X = balanced_df[['index', 'TVA (m3)', 'SPPA (kPa)', 'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)']]
balanced_y = balanced_df['STATUS']

# Use balanced_X and balanced_y for training your model

In [39]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Select the columns to normalize
columns_to_normalize = ['TVA (m3)', 'SPPA (kPa)', 'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)']  # Adjust the column names as per your requirement
# Perform the min-max normalization
balanced_df[columns_to_normalize]= scaler.fit_transform(balanced_df[columns_to_normalize])


array([[0.0848329 , 0.00655022, 0.1       , 0.        ],
       [0.0848329 , 0.00545852, 0.1       , 0.        ],
       [0.0848329 , 0.00545852, 0.1       , 0.        ],
       ...,
       [0.92544987, 0.73908297, 0.1       , 0.03630862],
       [0.92544987, 0.73580786, 0.08      , 0.03630862],
       [0.92544987, 0.73034934, 0.1       , 0.03570348]])

In [40]:
y=balanced_df['STATUS']
X=balanced_df[['TVA (m3)', 'SPPA (kPa)', 'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)']]


In [41]:
train_features , test_features , train_labels , test_labels = train_test_split(X , y, test_size=0.38, shuffle=False)
T = 36 # my choice of the rolling window
prepend_features = train_features.iloc[-(T- 1 ):]
test_features = pd.concat([prepend_features , test_features ], axis=0)
X_train , y_train = [], []
for i in range(train_labels.shape[ 0 ] - (T- 1 )):
    X_train.append(train_features.iloc[i:i+T].values)
    y_train.append(train_labels.iloc[i + (T- 1 )])
X_train , y_train = np.array(X_train), np.array(y_train).reshape(-1,1 )
print(f'Train_data_dimensions:{X_train.shape},{y_train.shape}')
X_test , y_test = [], []
for i in range(test_labels.shape[ 0 ]):
    X_test.append(test_features.iloc[i:i+T].values)
    y_test.append(test_labels.iloc[i])
X_test , y_test = np.array(X_test), np.array(y_test).reshape(- 1 , 1 )
print(f'Test_data_dimensions:{X_test.shape},{y_test.shape}')

Train_data_dimensions:(21257, 36, 4),(21257, 1)
Test_data_dimensions:(13050, 36, 4),(13050, 1)
